1.reading a csv file and creating a dataframe.
2.Applying transformation like, removing nulls, converting string type timestamp to time format.
3.Checking if any rows failed to convert into timestamps.
4.Filtering the rows that are not converted into timestamp.
5.Adding Date and Hours columns.
6.Finding the count of each action user did on each date.

In [0]:
df = spark.read.csv("/Volumes/workspace/default/sampledataset/user_activity_logs.csv", header = True, inferSchema = True)
df.display()
df.printSchema()

user_id,action,timestamp,category,device
1001,viewed,7/25/2025 10:12,Electronics,Mobile
1001,added_to_cart,7/25/2025 10:15,Electronics,Mobile
1001,purchased,7/25/2025 10:18,Electronics,Mobile
1002,viewed,7/25/2025 11:05,Books,Laptop
1002,viewed,7/25/2025 11:08,Books,Laptop
1003,viewed,7/25/2025 11:10,Clothing,Tablet


root
 |-- user_id: integer (nullable = true)
 |-- action: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- category: string (nullable = true)
 |-- device: string (nullable = true)



In [0]:
from pyspark.sql.functions import *

#drop nulls
df_clean = df.dropna()
df_clean = df_clean.withColumn("timestamp", to_timestamp("timestamp", "yyyy-MM-dd HH:mm:ss"))

# add date and hour columns
df_clean = df_clean\
.withColumn("date", to_date("timestamp"))\
.withColumn("hour", hour("timestamp"))
df_clean.display()



---------------------------------------------------------------------------
DateTimeException Traceback (most recent call last)
File , line 11
 7 # add date and hour columns
 8 df_clean = df_clean\
 9 .withColumn("date", to_date("timestamp"))\
 10 .withColumn("hour", hour("timestamp"))
---> 11 df_clean.display()

File /databricks/python_shell/lib/dbruntime/monkey_patches.py:67, in apply_dataframe_display_patch. .df_display(df, *args, **kwargs)
 63 def df_display(df, *args, **kwargs):
 64 """
 65 df.display() is an alias for display(df). Run help(display) for more information.
 66 """
---> 67 display(df, *args, **kwargs)

File /databricks/python_shell/lib/dbruntime/display.py:142, in Display.display(self, input, *args, **kwargs)
 140 # This version is for Serverless + Spark Connect dogfooding.
 141 elif self.spark_connect_enabled and isinstance(input, ConnectDataFrame):
--> 142 self.display_connect_table(input, **kwargs)
 143 elif isinstance(input, ConnectDataFrame):
 144 if input.isStreaming:

File /databricks/python_shell/lib/dbruntime/display.py:107, in Display.display_connect_table(self, df, **kwargs)
 104 self.cf_helper.display_streaming_dataframe(df, config, self.streaming_listener,
 105 **kwargs)
 106 else:
--> 107 self.cf_helper.display_dataframe(df, config)

File /databricks/python_shell/lib/dbruntime/display_helpers/cloudfetch_helper.py:72, in CloudFetchDisplayHelper.display_dataframe(self, df, config)
 71 def display_dataframe(self, df: ConnectDataFrame, config: TableDisplayConfig):
---> 72 display_payload = self.write_to_cloudfetch(df, config)
 73 ip_display({
 74 "application/vnd.databricks.connect.display": display_payload,
 75 "text/plain": "Databricks Spark Connect Table"
 76 },
 77 raw=True)

File /databricks/python_shell/lib/dbruntime/display_helpers/cloudfetch_helper.py:168, in CloudFetchDisplayHelper.write_to_cloudfetch(self, connectDataFrame, config)
 166 raise e
 167 except PySparkException as e:
--> 168 raise e
 169 except Exception as e:
 170 raise type(
 171 e
 172 )("Encountered an unexpected error when displaying table, please contact Databricks support"
 173 ) from e

File /databricks/python_shell/lib/dbruntime/display_helpers/cloudfetch_helper.py:132, in CloudFetchDisplayHelper.write_to_cloudfetch(self, connectDataFrame, config)
 128 # Check if the new cloudfetch API is available on the dataframe
 129 if CloudFetchDisplayHelper.use_pyspark_cloudfetch_api and hasattr(
 130 connectDataFrame, "_to_cloudfetch_with_limits_and_file_paths"):
 131 results: Tuple[Optional[StructType], List["CloudFetchResult"], List[
--> 132 bool]] = connectDataFrame._to_cloudfetch_with_limits_and_file_paths(
 133 format=str_format,
 134 compression=False,
 135 row_limit=rowLimit,
 136 byte_limit=byteLimit)
 137 pyspark_struct, cloudfetch_results, batch_truncation_results = results
 139 schema = CloudFetchDisplayHelper.get_display_schema_from_pyspark_struct(
 140 pyspark_struct)

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/dataframe.py:1963, in DataFrame._to_cloudfetch_with_limits_and_file_paths(self, format, compression, row_limit, byte_limit)
 1942 """
 1943 This is an experimental method to support generating results in the form of pre-signed
 1944 URLs that can be used by the client to fetch the results directly from cloud storage.
 (...)
 1960 
 1961 """
 1962 query = self._plan.to_proto(self._session.client)
-> 1963 return self._session.client.experimental_to_cloudfetch(
 1964 query, format, compression, row_limit, byte_limit, True
 1965 )

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/client/core.py:1083, in SparkConnectClient.experimental_to_cloudfetch(self, plan, format, compression, row_limit, byte_limit, with_internal_file_path)
 1080 # Execute the request and iterate over the streaming results. We capture the schema
 1081 # response and unpack the CloudResultBatch responses.
 1082 with Progress(handlers=self._progress_handlers, operation_id=req.operation_id) 

In [0]:
from pyspark.sql.functions import *

# Explicit format for timestamp (safe conversion)
# Your timestamp format is: "yyyy-MM-dd HH:mm:ss"
df_clean = df.withColumn(
    "timestamp_clean",
    to_timestamp("timestamp", "M/d/yyyy HH:mm")
)

# Check if any rows failed to convert
df_clean.select("timestamp", "timestamp_clean").show(truncate=False)

# Filter out rows where timestamp couldn't be parsed
df_clean = df_clean.filter(col("timestamp_clean").isNotNull())

# Add date and hour columns
df_clean = df_clean \
    .withColumn("date", to_date("timestamp_clean")) \
    .withColumn("hour", hour("timestamp_clean"))

display(df_clean)


+---------------+-------------------+
|timestamp      |timestamp_clean    |
+---------------+-------------------+
|7/25/2025 10:12|2025-07-25 10:12:00|
|7/25/2025 10:15|2025-07-25 10:15:00|
|7/25/2025 10:18|2025-07-25 10:18:00|
|7/25/2025 11:05|2025-07-25 11:05:00|
|7/25/2025 11:08|2025-07-25 11:08:00|
|7/25/2025 11:10|2025-07-25 11:10:00|
+---------------+-------------------+



user_id,action,timestamp,category,device,timestamp_clean,date,hour
1001,viewed,7/25/2025 10:12,Electronics,Mobile,2025-07-25T10:12:00.000Z,2025-07-25,10
1001,added_to_cart,7/25/2025 10:15,Electronics,Mobile,2025-07-25T10:15:00.000Z,2025-07-25,10
1001,purchased,7/25/2025 10:18,Electronics,Mobile,2025-07-25T10:18:00.000Z,2025-07-25,10
1002,viewed,7/25/2025 11:05,Books,Laptop,2025-07-25T11:05:00.000Z,2025-07-25,11
1002,viewed,7/25/2025 11:08,Books,Laptop,2025-07-25T11:08:00.000Z,2025-07-25,11
1003,viewed,7/25/2025 11:10,Clothing,Tablet,2025-07-25T11:10:00.000Z,2025-07-25,11


In [0]:
df_clean = df_clean.groupBy("user_id", "date", "action").agg(count("*").alias("action_count"))
display(df_clean)

user_id,date,action,action_count
1002,2025-07-25,viewed,2
1001,2025-07-25,purchased,1
1001,2025-07-25,viewed,1
1001,2025-07-25,added_to_cart,1
1003,2025-07-25,viewed,1


In [0]:
df_clean = df_clean.groupBy("user_id", "date")\
    .pivot('action', ['viewed', 'purchased', 'added_to_cart'])\
    .sum('action_count')\
    .fillna(0)
display(df_clean)


user_id,date,viewed,purchased,added_to_cart
1001,2025-07-25,1,1,1
1003,2025-07-25,1,0,0
1002,2025-07-25,2,0,0


In [0]:
df_clean.write.mode("overwrite").parquet("/Volumes/workspace/default/sampledataset/user_activity_logs")
#df_clean.write.format("delta").saveAsTable("user_activity_logs")

In [0]:
output_path = "/Volumes/workspace/default/sampledataset/user_activity_logs/"
print(f"✅ Final result written to: {output_path}")
display(dbutils.fs.ls(output_path))

✅ Final result written to: /Volumes/workspace/default/sampledataset/user_activity_logs/


path,name,size,modificationTime
dbfs:/Volumes/workspace/default/sampledataset/user_activity_logs/_SUCCESS,_SUCCESS,0,1754284974000
dbfs:/Volumes/workspace/default/sampledataset/user_activity_logs/_committed_5527395433473826661,_committed_5527395433473826661,124,1754284973000
dbfs:/Volumes/workspace/default/sampledataset/user_activity_logs/_started_5527395433473826661,_started_5527395433473826661,0,1754284973000
dbfs:/Volumes/workspace/default/sampledataset/user_activity_logs/part-00000-tid-5527395433473826661-45c8f626-ca11-44a2-9727-07e01c5b5295-136-1.c000.snappy.parquet,part-00000-tid-5527395433473826661-45c8f626-ca11-44a2-9727-07e01c5b5295-136-1.c000.snappy.parquet,1571,1754284973000
